In [180]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [122]:
data = pd.read_csv("./합격자 스펙 데이터_은행금융.csv", encoding = "euc-kr")
data

,Unnamed: 0,합격회사,기업정보,지원분야,출신전공,스펙지수,학점,토익,토익스피킹,OPIC,외국어(기타),자격증,해외경험,인턴,수상내역,교내/사회/봉사,종합정보,종목
0,1,(주)하나저축은행,"<span>은행·금융</span>,<span>959억 1,858만원</span>,<...",2018년 하반기 \n금융 보험영업,수도권4년제 어문학계열 중어중국학과,515,3.75/4.5,900점,Lv7,-,1개,-,2회,-,-,1회,"(주)하나저축은행<span>은행·금융</span>,<span>959억 1,858만원...","은행·금융,"
1,2,(주)하나저축은행,"<span>은행·금융</span>,<span>959억 1,858만원</span>,<...",2016년 하반기 \n기획 전략 경영,서울4년제 상경계열 경영계열,483,3.54/4.5,850점,-,IH,-,1개,-,-,1회,2회,"(주)하나저축은행<span>은행·금융</span>,<span>959억 1,858만원...","은행·금융,"
2,3,(주)하나저축은행,"<span>은행·금융</span>,<span>959억 1,858만원</span>,<...",2016년 하반기 \n사무보조 문서작성,"지방4년제 예/체능학계열 성악,뮤지컬",222,3.85/4.5,-,-,-,-,2개,-,-,-,-,"(주)하나저축은행<span>은행·금융</span>,<span>959억 1,858만원...","은행·금융,"
3,4,(주)하나저축은행,"<span>은행·금융</span>,<span>959억 1,858만원</span>,<...",2015년 상반기 \n비서 안내,대졸4년제 공학계열 컴퓨터학부,141,3.22/4.5,-,-,-,-,-,-,-,-,-,"(주)하나저축은행<span>은행·금융</span>,<span>959억 1,858만원...","은행·금융,"
4,5,(주)하나저축은행,"<span>은행·금융</span>,<span>959억 1,858만원</span>,<...",2013년 하반기 \n금융 보험영업,지방4년제 법학계열 법경찰,203,3/4.5,-,-,-,-,2개,-,-,-,-,"(주)하나저축은행<span>은행·금융</span>,<span>959억 1,858만원...","은행·금융,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32775,32776,우리신용정보㈜,"<span>은행·금융</span>,<span>395억 6,765만원</span>,<...",2007년 상반기 \n경리 회계 결산,초대졸 정보통신,83,-,-,-,-,-,1개,-,-,-,-,"우리신용정보㈜<span>은행·금융</span>,<span>395억 6,765만원</...","은행·금융,"
32776,32777,우리신용정보㈜,"<span>은행·금융</span>,<span>395억 6,765만원</span>,<...",2007년 상반기 \n채권 심사 보험 보상,고졸,30,-,-,-,-,-,-,-,-,-,-,"우리신용정보㈜<span>은행·금융</span>,<span>395억 6,765만원</...","은행·금융,"
32777,32778,우리신용정보㈜,"<span>은행·금융</span>,<span>395억 6,765만원</span>,<...",2007년 상반기 \n고객상담 인바운드,수도권4년제 예/체능학계열 연극,162,4.16/4.5,-,-,-,-,-,-,-,-,-,"우리신용정보㈜<span>은행·금융</span>,<span>395억 6,765만원</...","은행·금융,"
32778,32779,우리신용정보㈜,"<span>은행·금융</span>,<span>395억 6,765만원</span>,<...",2007년 상반기 \n채권 심사 보험 보상,지방4년제 상경계열 무역,136,3/4.5,-,-,-,-,-,-,-,-,-,"우리신용정보㈜<span>은행·금융</span>,<span>395억 6,765만원</...","은행·금융,"


In [177]:
data_modi = data.copy()

import re
data_modi = data_modi.drop("Unnamed: 0",axis=1)
companySize = []
for i in range (0,data_modi.shape[0]) : 
    #지원정보 단순화
    data_modi.iloc[i,2] = data_modi.iloc[i,2][11:99]
    #기업정보 단순화 (참고로 매출액 400억원 이하가 중소기업, 80억원 이하가 소기업, 대기업은 매출액 5조 이상?)
    ## 기업 규모 추출
    imsi = re.split("</span>,<span>",data.iloc[i,2])[1].replace(",","")
    if ("조" in imsi) & ("억" in imsi) & ("만원" in imsi) :
        imsi2 = re.split("조|억|만원",imsi)[0:3]
        #숫자로 바꾸기
        total = int(imsi2[0])*(10**12)+int(imsi2[1])*(10**8)+int(imsi2[2])*(10**4)
        companySize.append(total)
    elif ("억" in imsi) & ("만원" in imsi) : 
        imsi2 = re.split("억|만원",imsi)[0:2]
        total = int(imsi2[0])*(10**8)+int(imsi2[1])*(10**4)
        companySize.append(total)
    elif ("억" in imsi) & ("만" in imsi) : 
        imsi2 = re.split("억|만",imsi)[0:2]
        total = int(imsi2[0])*(10**8)+int(imsi2[1])*(10**4)
        companySize.append(total)
    elif ("조" in imsi) & ("억천" in imsi) : #억천을 그냥 억으로 취급함
        imsi2 = re.split("조|억천",imsi)[0:2]
        total = int(imsi2[0])*(10**12)+int(imsi2[1])*(10**8)
        companySize.append(total)
        companySizeisNull.append(0)
    elif ("조" in imsi) & ("억원" in imsi) : #억천을 그냥 억으로 취급함
        imsi2 = re.split("조|억원",imsi)[0:2]
        total = int(imsi2[0])*(10**12)+int(imsi2[1])*(10**8)
        companySize.append(total)
    elif ("조" in imsi) & ("억" in imsi) : #억천을 그냥 억으로 취급함
        imsi2 = re.split("조|억",imsi)[0:2]
        total = int(imsi2[0])*(10**12)+int(imsi2[1])*(10**8)
        companySize.append(total)
    elif ("천억" in imsi) :
        imsi2 = re.split("천억",imsi)[0:2]
        total = int(imsi2[0])*(10**8)*(1000)
        companySize.append(total)
    elif ("억" in imsi) :
        imsi2 = re.split("억",imsi)[0:2]
        total = int(imsi2[0])*(10**8)
        companySize.append(total)
    else : #기업 규모액이 없음
        companySize.append(0)
    ##학력 컨버팅
    if ("4년제" in data_modi.iloc[i,3]):
        data_modi.iloc[i,3] = 2  
    elif ("대학원" in data_modi.iloc[i,3]) : 
        data_modi.iloc[i,3] = 3
    elif ("해외대학원" in data_modi.iloc[i,3]) : 
        data_modi.iloc[i,3] = 3
    elif ("해외대학" in data_modi.iloc[i,3]) : 
        data_modi.iloc[i,3] = 2
    elif ("초대졸" in data_modi.iloc[i,3]) : 
        data_modi.iloc[i,3] = 1
    elif ("고졸" in data_modi.iloc[i,3]) : 
        data_modi.iloc[i,3] = 0
    
    ##학점 컨버팅
    if (data_modi.iloc[i,5] != "-") :
        data_modi.iloc[i,5] = float(data_modi.iloc[i,5].replace("/4.5",""))    
    else :
        data_modi.iloc[i,5] = 0
    
    ##토익 컨버팅
    if (data_modi.iloc[i,6] != "-") :
        data_modi.iloc[i,6] = int(data_modi.iloc[i,6].replace("점",""))    
    else :
        data_modi.iloc[i,6] = 0
    ##토익 스피킹 컨버팅
    if (data_modi.iloc[i,7] == "Lv1") :
        data_modi.iloc[i,7] = 1
    elif (data_modi.iloc[i,7] == "Lv2") :
        data_modi.iloc[i,7] = 2
    elif (data_modi.iloc[i,7] == "Lv3") :
        data_modi.iloc[i,7] = 3
    elif (data_modi.iloc[i,7] == "Lv4") :
        data_modi.iloc[i,7] = 4
    elif (data_modi.iloc[i,7] == "Lv5") :
        data_modi.iloc[i,7] = 5
    elif (data_modi.iloc[i,7] == "Lv6") :
        data_modi.iloc[i,7] = 6
    elif (data_modi.iloc[i,7] == "Lv7") :
        data_modi.iloc[i,7] = 7
    elif (data_modi.iloc[i,7] == "Lv8") :
        data_modi.iloc[i,7] = 8
    else :
        data_modi.iloc[i,7] = 0

    ##오픽 컨버팅
    if (data_modi.iloc[i,8] == "NL") :
        data_modi.iloc[i,8] = 1
    elif (data_modi.iloc[i,8] == "NM") :
        data_modi.iloc[i,8] = 2
    elif (data_modi.iloc[i,8] == "NH") :
        data_modi.iloc[i,8] = 3
    elif (data_modi.iloc[i,8] == "IL") :
        data_modi.iloc[i,8] = 4
    elif (data_modi.iloc[i,8] == "IM1") :
        data_modi.iloc[i,8] = 5
    elif (data_modi.iloc[i,8] == "IM2") :
        data_modi.iloc[i,8] = 6
    elif (data_modi.iloc[i,8] == "IM3") :
        data_modi.iloc[i,8] = 7
    elif (data_modi.iloc[i,8] == "IH") :
        data_modi.iloc[i,8] = 8
    elif (data_modi.iloc[i,8] == "AL") :
        data_modi.iloc[i,8] = 9
    else :
        data_modi.iloc[i,8] = 0
        
    ##제2외국어 컨버팅
    if (data_modi.iloc[i,9] != "-") :
        data_modi.iloc[i,9] = int(data_modi.iloc[i,9].replace("개",""))    
    else :
        data_modi.iloc[i,9] = 0
        
    ##자격증 컨버팅
    if (data_modi.iloc[i,10] != "-") :
        data_modi.iloc[i,10] = int(data_modi.iloc[i,10].replace("개",""))    
    else :
        data_modi.iloc[i,10] = 0

    ##해외경험 컨버팅
    if (data_modi.iloc[i,11] != "-") :
        data_modi.iloc[i,11] = int(data_modi.iloc[i,11].replace("회",""))    
    else :
        data_modi.iloc[i,11] = 0
        
    ##인턴 컨버팅
    if (data_modi.iloc[i,12] != "-") :
        data_modi.iloc[i,12] = int(data_modi.iloc[i,12].replace("회",""))    
    else :
        data_modi.iloc[i,12] = 0

    ##수상내역 컨버팅
    if (data_modi.iloc[i,13] != "-") :
        data_modi.iloc[i,13] = int(data_modi.iloc[i,13].replace("회",""))    
    else :
        data_modi.iloc[i,13] = 0

    ##사회봉사 컨버팅
    if (data_modi.iloc[i,14] != "-") :
        data_modi.iloc[i,14] = int(data_modi.iloc[i,14].replace("회",""))    
    else :
        data_modi.iloc[i,14] = 0
    
        
data_modi.insert(2,"기업 총액(원)",companySize)
data_modi = data_modi.drop("기업정보",axis=1)
data_modi = data_modi.drop("종합정보",axis=1)
data_modi = data_modi.drop("종목",axis=1)


data_modi


,합격회사,기업 총액(원),지원분야,출신전공,스펙지수,학점,토익,토익스피킹,OPIC,외국어(기타),자격증,해외경험,인턴,수상내역,교내/사회/봉사
0,(주)하나저축은행,95918580000,금융 보험영업,2,515,3.75,900,7,0,1,0,2,0,0,1
1,(주)하나저축은행,95918580000,기획 전략 경영,2,483,3.54,850,0,8,0,1,0,0,1,2
2,(주)하나저축은행,95918580000,사무보조 문서작성,2,222,3.85,0,0,0,0,2,0,0,0,0
3,(주)하나저축은행,95918580000,비서 안내,2,141,3.22,0,0,0,0,0,0,0,0,0
4,(주)하나저축은행,95918580000,금융 보험영업,2,203,3,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32775,우리신용정보㈜,39567650000,경리 회계 결산,1,83,0,0,0,0,0,1,0,0,0,0
32776,우리신용정보㈜,39567650000,채권 심사 보험 보상,0,30,0,0,0,0,0,0,0,0,0,0
32777,우리신용정보㈜,39567650000,고객상담 인바운드,2,162,4.16,0,0,0,0,0,0,0,0,0
32778,우리신용정보㈜,39567650000,채권 심사 보험 보상,2,136,3,0,0,0,0,0,0,0,0,0


In [205]:
X = data_modi.iloc[:,[3,5,6,7,8,9,10,11,12,13,14]].copy()
y = data_modi.iloc[:,0].copy()
y2 = pd.get_dummies(y)
X
y

0        (주)하나저축은행
1        (주)하나저축은행
2        (주)하나저축은행
3        (주)하나저축은행
4        (주)하나저축은행
           ...    
32775      우리신용정보㈜
32776      우리신용정보㈜
32777      우리신용정보㈜
32778      우리신용정보㈜
32779      우리신용정보㈜
Name: 합격회사, Length: 32780, dtype: object

In [217]:
#라벨 인코딩

y_label = y.copy()
indexdata = range(0,len(y_label.unique()))
imsiSeries = pd.Series(data=indexdata, index=y_label.unique())
y_label = y_label.map(imsiSeries)
indexdata
y_label

0          0
1          0
2          0
3          0
4          0
        ... 
32775    118
32776    118
32777    118
32778    118
32779    118
Name: 합격회사, Length: 32780, dtype: int64

In [220]:
# 기업 매출액으로 대중소기업 나누기
y_money = y.copy()
for i in range (0,data_modi.shape[0]) : 
    if (data_modi.iloc[i,1] >= 5000000000000 ) : #대기업, 5조
        y_money.iloc[i] = 4
    elif (data_modi.iloc[i,1] >= 40000000000 ) : #중견기업, 400억
        y_money.iloc[i] = 3
    elif (data_modi.iloc[i,1] == 0) : #데이터 없음
        y_money.iloc[i] = 2
    elif (data_modi.iloc[i,1] >= 8000000000) : #중소기업, 80억 이상
        y_money.iloc[i] = 1
    else :
        y_money.iloc[i] = 0

y_money

0        3
1        3
2        3
3        3
4        3
        ..
32775    1
32776    1
32777    1
32778    1
32779    1
Name: 합격회사, Length: 32780, dtype: object

In [253]:
y_money = y_money.astype(int)

In [254]:
#트레인셋 나누기

X_train, X_test, y_train, y_test = train_test_split(X,y_money,random_state = 2, test_size=0.3)

X_train

,출신전공,학점,토익,토익스피킹,OPIC,외국어(기타),자격증,해외경험,인턴,수상내역,교내/사회/봉사
24503,0,0,0,0,0,0,3,0,0,0,0
24150,2,2.2,0,0,0,0,0,0,0,0,0
2969,2,3.7,870,0,0,0,3,0,0,0,0
8432,2,3.17,0,0,0,1,1,1,0,0,0
31763,2,0,820,0,0,0,4,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
18898,0,0,0,0,0,0,0,0,0,0,0
31019,1,3.3,0,0,0,0,0,0,0,2,0
30280,1,3.8,0,0,0,0,0,0,0,0,0
6637,1,4.1,0,0,0,0,0,0,0,0,0


In [255]:
knn = KNeighborsClassifier(n_neighbors = 3)

In [256]:
knn.fit(X_train,y_train)

y_predict = knn.predict(X_test)

In [257]:
knn.score(X_train,y_train)

0.6443388825939161

In [287]:
#스펙 점수로 해볼까?
X_spec_imsi = data_modi.iloc[:,4].copy()
X_spec = data_modi.iloc[:,[4,5]].copy()
X_spec.insert(0,"스펙 점수 임시",X_spec_imsi)

X_spec = X_spec.drop("학점",axis=1)

X_spec

,스펙 점수 임시,스펙지수
0,515,515
1,483,483
2,222,222
3,141,141
4,203,203
...,...,...
32775,83,83
32776,30,30
32777,162,162
32778,136,136


In [293]:
X_spec_1d = X_spec.copy()
X_spec_1d = X_spec_1d.drop("스펙 점수 임시",axis=1)
X_spec_1d

,스펙지수
0,515
1,483
2,222
3,141
4,203
...,...
32775,83
32776,30
32777,162
32778,136


In [294]:
knn_spec = KNeighborsClassifier(n_neighbors = 3)

X_train, X_test, y_train, y_test = train_test_split(X_spec_1d,y_money,random_state = 2, test_size=0.3)

In [296]:
knn_spec.fit(X_train,y_train)

y_predict = knn_spec.predict(X_test)

knn_spec.score(X_train,y_train)

0.5195241000610128